In [1]:
# pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
# !pip install scipy

In [3]:
import os
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [4]:
# 把圖片/文案和目標值擷取出來
import os
import pandas as pd

csv_file = 'data_new.csv'
img_folder = 'photos/'

# 讀取CSV檔案
df = pd.read_csv('data_new.csv')

# 創建空列表
imgs = []
good_or_bad = []
text=[]

# 對圖像文件夾進行迭代
for img_name in os.listdir(img_folder):
    # 取得圖片ID
    img_id = int(img_name.split('.')[0])
    # 從csv中取得1或0
    g_o_b = df[df['ID']==img_id]['good_or_bad_pics'].values[0]
    Text = df[df['ID']==img_id]['Text'].values[0]
    # 將圖片和曝光數值分別加入列表中
    imgs.append(img_name)
    good_or_bad.append(g_o_b)
    text.append(Text)

# print(imgs)
# print(good_or_bad)
# print(text)

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 文案轉向量
def extract_text_features(texts):
    
    # 將含有 NaN 的值替換成空字符串
    text = [str(t) if pd.notnull(t) else '' for t in texts]
    
    # 建立CountVectorizer和TfidfVectorizer物件
    # count_vect = CountVectorizer()
    tfidf_vect = TfidfVectorizer()

    # 將列表中的文字轉換成向量
    # count_vectors = count_vect.fit_transform(text)
    tfidf_vectors = tfidf_vect.fit_transform(text)

    # 將稀疏矩陣轉換為NumPy陣列
    # count_vectors = count_vectors.toarray()
    tfidf_vectors = tfidf_vectors.toarray()

    return tfidf_vectors
#     return count_vectors


In [6]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from PIL import Image

# 載入 VGG16 模型
model = VGG16(weights='imagenet', include_top=False)

# 使用 VGG16 提取圖像特徵
def extract_image_features(img_path):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    x = np.array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.reshape(-1,) 


In [7]:
from sklearn.ensemble import RandomForestClassifier

# 假設 X_image 和 X_text 分別代表圖片和文案的向量表示，y 則代表對應的標籤
img_dir = "photos"
X_image = np.array([extract_image_features(os.path.join(img_dir, f)) for f in os.listdir(img_dir)])
X_text = extract_text_features(text)
y = good_or_bad

# 將圖片和文案的向量合併為一個輸入
X = np.hstack((X_image, X_text))

# 使用隨機森林進行訓練
rf = RandomForestClassifier()
rf.fit(X, y)

# 計算圖片和文案對應的權重分數
image_importance = rf.feature_importances_[:X_image.shape[1]]
text_importance = rf.feature_importances_[X_image.shape[1]:]
print("圖片特徵權重分數：", image_importance)
print("文案特徵權重分數：", text_importance)

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 22ms/step
圖片特徵權重分數： [0.00000000e+00 0.00000000e+00 9.69123182e-05 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
文案特徵權重分數： [0. 0. 0. ... 0. 0. 0.]


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 創建隨機森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 預測測試集
y_pred = rf_model.predict(X_test)

# 評估模型準確度
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5416666666666666


In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = np.hstack((X_image, X_text))
y = good_or_bad
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
# 創建隨機森林模型
rf = RandomForestRegressor()
# 訓練模型
rf.fit(X_train, y_train)

# 預測測試集
y_pred = rf.predict(X_test)

# 計算 R2 分數
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

R2 Score: -0.018101730019493223
